# Check DC2 Files

- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2024-11-03
- last update :  2024-11-03


In [ ]:
from rail.utils.path_utils import find_rail_file
import h5py
import pandas as pd
import numpy as np
import astropy
from astropy import units as u
from astropy import constants as c
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# reference flux in Jy
F0 = ((0.0 * u.ABmag).to(u.Jy)).value
F0

## Config

In [ ]:
trainFile = find_rail_file("examples_data/testdata/test_dc2_training_9816.hdf5")
testFile = find_rail_file("examples_data/testdata/test_dc2_validation_9816.hdf5")

In [ ]:
# the order by which one want the data
list_of_cols = [
    "id",
    "redshift",
    "mag_u_lsst",
    "mag_g_lsst",
    "mag_r_lsst",
    "mag_i_lsst",
    "mag_z_lsst",
    "mag_y_lsst",
    "mag_err_u_lsst",
    "mag_err_g_lsst",
    "mag_err_r_lsst",
    "mag_err_i_lsst",
    "mag_err_z_lsst",
    "mag_err_y_lsst",
]
list_of_filters = ["u", "g", "r", "i", "z", "y"]
list_of_fitcolors = ["b", "g", "r", "orange", "grey", "k"]
Nf = len(list_of_filters)

## Read magnitudes file in pandas dataframe

In [ ]:
def h5filetodataframe(filename, group="photometry"):
    """
    Function to convert the LSST magnitudes hdf5 file into a pandas dataFrame
    """
    data = h5py.File(filename, "r")
    list_of_keys = list(data[group].keys())
    all_data = np.array([data[group][key][:] for key in list_of_keys])
    df = pd.DataFrame(all_data.T, columns=list_of_keys)
    if "id" in list_of_keys:
        df = df.astype({"id": int})
    return df

In [ ]:
df_train = h5filetodataframe(trainFile)
df_test = h5filetodataframe(testFile)

In [ ]:
df_train = df_train[list_of_cols]
df_test = df_test[list_of_cols]

In [ ]:
df_train

In [ ]:
df_train.isnull().values.any()

In [ ]:
df_test

In [ ]:
df_test.isnull().values.any()

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

## Convert 

In [ ]:
def CheckBadFluxes(fl, dfl, mag, dmag, maxmag=30.0):
    """ """
    indexes_bad = np.where(mag > maxmag)[0]
    indexes_good = np.where(mag < maxmag)[0]

    if len(indexes_bad) > 0:
        for idx in indexes_bad:
            # in band g,r,i,z
            if idx > 0 and idx < Nf - 1:
                # have two good neighbourgs
                if idx - 1 in indexes_good and idx + 1 in indexes_good:
                    fl[idx] = np.mean([fl[idx - 1], fl[idx + 1]])
                    dfl[idx] = np.max([dfl[idx - 1], dfl[idx + 1]]) * 5.0
                elif idx - 1 in indexes_good:
                    fl[idx] = fl[idx - 1]
                    dfl[idx] = dfl[idx - 1] * 10.0
                elif idx + 1 in indexes_good:
                    fl[idx] = fl[idx + 1]
                    dfl[idx] = dfl[idx + 1] * 10.0
                else:
                    fl[idx] = np.mean(fl[indexes_good])
                    dfl[idx] = np.max(fl[indexes_good]) * 100.0
            elif idx == 0:
                if idx + 1 in indexes_good:
                    fl[idx] = fl[idx + 1]
                    dfl[idx] = dfl[idx + 1] * 10.0
                else:
                    fl[idx] = np.mean(fl[indexes_good])
                    dfl[idx] = np.max(fl[indexes_good]) * 100.0
            elif idx == Nf - 1:
                if idx - 1 in indexes_good:
                    fl[idx] = fl[idx - 1]
                    dfl[idx] = dfl[idx - 1] * 10.0
                else:
                    fl[idx] = np.mean(fl[indexes_good])
                    dfl[idx] = np.max(fl[indexes_good]) * 100.0

    return fl, dfl

In [ ]:
def convert_to_ABflux(row):
    """
    Convert AB magnitudes into FAB flux (units AB, that is per 3631 Jy
    """

    fl = np.zeros(Nf)
    dfl = np.zeros(Nf)
    mag = np.zeros(Nf)
    dmag = np.zeros(Nf)
    all_fname = []
    all_ferrname = []

    for idx, band in enumerate(list_of_filters):
        mag_label = f"mag_{band}_lsst"
        magerr_label = f"mag_err_{band}_lsst"
        flux_label = f"fab_{band}_lsst"
        fluxerr_label = f"fab_err_{band}_lsst"
        m = row[mag_label]
        dm = row[magerr_label]
        f = np.power(10.0, -0.4 * m)
        df = np.log(10.0) / 2.5 * f * dm
        fl[idx] = f
        mag[idx] = m
        dfl[idx] = df
        dmag[idx] = dm
        all_fname.append(flux_label)
        all_ferrname.append(fluxerr_label)

    # decide what to do if one magnitude is too high
    fl, dfl = CheckBadFluxes(fl, dfl, mag, dmag)
    column_names = all_fname + all_ferrname
    data = np.concatenate((fl, dfl))
    return pd.Series(data, index=column_names)

In [ ]:
df_train_fl = df_train.apply(convert_to_ABflux, axis=1)

In [ ]:
df_test_fl = df_test.apply(convert_to_ABflux, axis=1)

In [ ]:
df_train_fl.isnull().values.any()

In [ ]:
df_test_fl.isnull().values.any()

In [ ]:
flux_col = [f"fab_{band}_lsst" for band in list_of_filters]
eflux_col = [f"fab_err_{band}_lsst" for band in list_of_filters]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

for ifilt in range(Nf):
    tag = flux_col[ifilt]
    xval = -2.5 * np.log10(df_test_fl[tag])
    legname = list_of_filters[ifilt]
    ax.hist(xval, bins=100, histtype="step", lw=3, color=list_of_fitcolors[ifilt], label=legname)
ax.set_xlabel("-2.5*log_10(Flux) in AB unit")
ax.set_ylabel("nb of galaxies")
ax.set_title("Distribution of Fluxes")
ax.legend(loc="upper right")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

for ifilt in range(Nf):
    tag = flux_col[ifilt]
    etag = eflux_col[ifilt]
    xval = -2.5 * np.log10(df_test_fl[tag])
    yval = -2.5 * np.log10(df_test_fl[etag])
    legname = list_of_filters[ifilt]
    plt.scatter(xval, yval, marker=".", color=list_of_fitcolors[ifilt], label=legname)
ax.set_xlabel("-2.5*log_10(Flux) in AB unit")
ax.set_ylabel("-2.5*log_10(Flux_Err) in AB unit")
ax.set_title("Flux err vs Fluxes")
ax.legend(loc="upper right")